<a href="https://colab.research.google.com/github/VivekSaini11/Machine-Learning-Project/blob/master/Mercedes_Benz_Greener_Manufacturing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**DESCRIPTION**

Reduce the time a Mercedes-Benz spends on the test bench.

Problem Statement Scenario:
Since the first automobile, the Benz Patent Motor Car in 1886, Mercedes-Benz has stood for important automotive innovations. These include the passenger safety cell with a crumple zone, the airbag, and intelligent assistance systems. Mercedes-Benz applies for nearly 2000 patents per year, making the brand the European leader among premium carmakers. Mercedes-Benz is the leader in the premium car industry. With a huge selection of features and options, customers can choose the customized Mercedes-Benz of their dreams.

To ensure the safety and reliability of every unique car configuration before they hit the road, the company’s engineers have developed a robust testing system. As one of the world’s biggest manufacturers of premium cars, safety and efficiency are paramount on Mercedes-Benz’s production lines. However, optimizing the speed of their testing system for many possible feature combinations is complex and time-consuming without a powerful algorithmic approach.

You are required to reduce the time that cars spend on the test bench. Others will work with a dataset representing different permutations of features in a Mercedes-Benz car to predict the time it takes to pass testing. Optimal algorithms will contribute to faster testing, resulting in lower carbon dioxide emissions without reducing Mercedes-Benz’s standards.

Firstly we are importing all the nessary libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from xgboost.sklearn import XGBRegressor

Here we are importing training and testing data.

In [ ]:
#importing the data.
train_data=pd.read_csv('train.csv');
test_data=pd.read_csv('test.csv');

Here we are going to take a look of training and testing data

In [ ]:
train_data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Here we are checking the variance of the training and testing data. variance is not equal to zero so that no need to remove the varible from the dataset.

In [ ]:
# If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
train_data.var()

ID      1.476658e+06
y       4.817973e+05
X0      2.316842e+02
X1      7.300882e+01
X2      1.045980e+02
            ...     
X380    8.014579e-03
X382    8.715481e-03
X383    4.750593e-04
X384    7.124196e-04
X385    1.660732e-03
Length: 378, dtype: float64

In [ ]:
test_data.var()

ID      1.476658e+06
X0      2.316842e+02
X1      7.300882e+01
X2      1.045980e+02
X3      3.157646e+00
            ...     
X380    8.014579e-03
X382    8.715481e-03
X383    4.750593e-04
X384    7.124196e-04
X385    1.660732e-03
Length: 377, dtype: float64

Here we are checking for shape of training and testing data

In [ ]:
train_data.shape

(4209, 378)

In [ ]:
test_data.shape

(4209, 377)

Here we are checking for the null values in training and testing data.

In [ ]:
# Check for null and unique values for test and train sets
train_data.isnull().sum()

ID      0
y       0
X0      0
X1      0
X2      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 378, dtype: int64

In [ ]:
test_data.isnull().sum()

ID      0
X0      0
X1      0
X2      0
X3      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 377, dtype: int64

Checking for type of datatypes 

In [ ]:
train_data.dtypes

ID        int64
y       float64
X0       object
X1       object
X2       object
         ...   
X380      int64
X382      int64
X383      int64
X384      int64
X385      int64
Length: 378, dtype: object

In [ ]:
test_data.dtypes

ID      int64
X0      int64
X1      int64
X2      int64
X3      int64
        ...  
X380    int64
X382    int64
X383    int64
X384    int64
X385    int64
Length: 377, dtype: object

Here we have converted the catagorical data of trainig and testing into the encoded form using LabelEncoder of sklearn library.

In [ ]:
# Apply label encoder for train data.
categorical_feature_mask = train_data.dtypes == object
categorical_cols = train_data.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
for col in train_data.columns:
    train_data[col] = le.fit_transform(train_data[col])
X_train_features = train_data.drop(['y'], axis=1)
y_train_label = train_data.iloc[:, 1]
X_train_features

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,21,23,34,5,3,26,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,42,3,8,0,3,9,6,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,21,23,17,5,3,0,9,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,21,13,34,5,3,31,11,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4,45,20,17,2,3,30,8,12,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,4204,6,9,17,5,3,1,9,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4205,4205,42,1,8,3,3,1,9,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,4206,47,23,17,5,3,1,3,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4207,4207,7,23,17,0,3,1,2,16,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
# Apply label encoder for test data.
categorical_feature_mask = test_data.dtypes == object
categorical_cols = test_data.columns[categorical_feature_mask].tolist()
le = LabelEncoder()
for col in test_data.columns:
    test_data[col] = le.fit_transform(test_data[col])
test_data

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,...,X345,X346,X347,X348,X349,X350,X351,X352,X353,X354,X355,X356,X357,X358,X359,X360,X361,X362,X363,X364,X365,X366,X367,X368,X369,X370,X371,X372,X373,X374,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,21,23,34,5,3,26,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,42,3,8,0,3,9,6,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,2,21,23,17,5,3,0,9,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,21,13,34,5,3,31,11,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,4,45,20,17,2,3,30,8,12,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,4204,6,9,17,5,3,1,9,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4205,4205,42,1,8,3,3,1,9,24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4206,4206,47,23,17,5,3,1,3,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4207,4207,7,23,17,0,3,1,2,16,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


Here we are performing the dimentionality reduction using PCA for both trainig and testing data.

In [ ]:
# Perform dimensionality reduction for train data.
pca = PCA()
fit = pca.fit(X_train_features)
print(("Explained Variance: %s") % (fit.explained_variance_ratio_))
fit.components_

Explained Variance: [9.99659608e-01 1.38083753e-04 7.69770342e-05 4.40107808e-05
 3.31849481e-05 2.66009200e-05 5.73061934e-06 2.67845282e-06
 1.56203752e-06 1.05802559e-06 8.69673184e-07 8.51175070e-07
 7.32194816e-07 6.01103751e-07 5.33242231e-07 4.63034402e-07
 3.72740073e-07 3.44879436e-07 3.21997035e-07 2.83068830e-07
 2.55585326e-07 2.31969263e-07 2.24009902e-07 2.10430778e-07
 1.84605148e-07 1.73951010e-07 1.55695102e-07 1.49112506e-07
 1.35988598e-07 1.32568757e-07 1.28164240e-07 1.18615786e-07
 1.13303372e-07 1.07956082e-07 1.01596937e-07 9.63933157e-08
 8.94341956e-08 7.97400199e-08 7.72270274e-08 7.44833379e-08
 7.07897886e-08 6.70931168e-08 6.42211701e-08 6.33394701e-08
 6.28443767e-08 6.08236279e-08 5.50354934e-08 5.38430316e-08
 5.17805622e-08 5.09377636e-08 4.97711458e-08 4.80779646e-08
 4.66612734e-08 4.53820263e-08 4.29839488e-08 4.12915804e-08
 3.88307962e-08 3.85021592e-08 3.70761370e-08 3.65157851e-08
 3.55486692e-08 3.34676588e-08 3.26009107e-08 3.13128247e-08
 3.0

In [ ]:
train_data_pca = pca.transform(X_train_features)
train_data_pca

array([[ 2.10391944e+03,  1.48901578e+01,  1.43106871e+01, ...,
        -2.02462109e-17, -6.19292286e-18,  8.04634137e-14],
       [ 2.10300998e+03, -1.48068922e+01, -8.10199579e+00, ...,
         1.71337726e-17,  6.25692498e-18, -4.15417700e-13],
       [ 2.10203711e+03,  1.23521650e+01, -2.18928463e+00, ...,
         8.88616544e-18,  7.62712644e-19,  3.93574062e-13],
       ...,
       [-2.10191831e+03, -1.37536955e+01,  3.21752326e+00, ...,
        -2.68036439e-19, -9.09209592e-20, -6.78415657e-14],
       [-2.10292321e+03,  2.46245283e+01, -5.02830129e+00, ...,
         1.00097829e-19,  6.57053182e-20,  1.83880688e-15],
       [-2.10390525e+03, -1.56684906e+01, -7.94246831e+00, ...,
        -4.49107204e-20,  1.98079181e-20,  3.88578059e-16]])

In [ ]:
# Perform dimensionality reduction for test data.
pca = PCA()
fit = pca.fit(test_data)
print(("Explained Variance: %s") % (fit.explained_variance_ratio_))
print(fit.components_)

Explained Variance: [9.99637654e-01 1.67216554e-04 6.79028030e-05 4.33597184e-05
 3.31749814e-05 2.93170202e-05 5.51112705e-06 2.77341533e-06
 1.56251928e-06 1.04907502e-06 8.76283729e-07 8.49894166e-07
 7.02009031e-07 6.05625404e-07 5.32076043e-07 4.64258869e-07
 3.82358002e-07 3.42739156e-07 3.16267727e-07 2.78013508e-07
 2.52085650e-07 2.39597335e-07 2.21370006e-07 2.08158236e-07
 1.87777755e-07 1.66944469e-07 1.58877812e-07 1.47068245e-07
 1.39250539e-07 1.33921443e-07 1.31242935e-07 1.21100280e-07
 1.10701213e-07 1.08502992e-07 1.01950087e-07 9.17297112e-08
 9.05663968e-08 8.40070272e-08 8.04864409e-08 7.34261933e-08
 6.81349579e-08 6.65158655e-08 6.53987986e-08 6.17668613e-08
 6.01522389e-08 5.69797553e-08 5.37882278e-08 5.33147814e-08
 5.21305542e-08 5.06544661e-08 4.97423227e-08 4.72272670e-08
 4.57902852e-08 4.34656188e-08 4.22184584e-08 4.20057991e-08
 3.80805902e-08 3.74818188e-08 3.68631713e-08 3.50352268e-08
 3.47547055e-08 3.35025458e-08 3.30300110e-08 3.18722195e-08
 3.0

In [ ]:
test_data_pca = pca.transform(test_data)
test_data_pca

array([[ 2.10391944e+03,  1.48901578e+01,  1.43106871e+01, ...,
        -2.02462109e-17, -6.19292286e-18,  8.04634137e-14],
       [ 2.10300998e+03, -1.48068922e+01, -8.10199579e+00, ...,
         1.71337726e-17,  6.25692498e-18, -4.15417700e-13],
       [ 2.10203711e+03,  1.23521650e+01, -2.18928463e+00, ...,
         8.88616544e-18,  7.62712644e-19,  3.93574062e-13],
       ...,
       [-2.10191831e+03, -1.37536955e+01,  3.21752326e+00, ...,
        -2.68036439e-19, -9.09209592e-20, -6.78415657e-14],
       [-2.10292321e+03,  2.46245283e+01, -5.02830129e+00, ...,
         1.00097829e-19,  6.57053182e-20,  1.83880688e-15],
       [-2.10390525e+03, -1.56684906e+01, -7.94246831e+00, ...,
        -4.49107204e-20,  1.98079181e-20,  3.88578059e-16]])

Now finally we are going to predict the test data labels.Because here we do not have labels for the test data So here we can not check the accuracy of the model but simply predict the test data labels.

In [ ]:
# Predict your test_df values using XGBoost.
xgb_model = XGBRegressor(objective="reg:linear", random_state=42)
xgb_model.fit(train_data_pca, y_train_label)
y_pred = xgb_model.predict(test_data_pca)
y_pred

[11:21:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


array([1732.7095 ,  994.4461 ,  668.6674 , ..., 1019.84235, 1107.7944 ,
       1337.6077 ], dtype=float32)